In [ ]:
from currensee.agents.complete_graph import compiled_graph
from currensee.agents.tools.finance_tools import generate_macro_table
import warnings

warnings.filterwarnings("ignore")

## Define Initial State

This is data that we should be retrieving from each meeting invite.

**DO NOT** change this data until the CRM DB has been updated

In [ ]:
init_state = {
    "client_name": "Adam Clay",
    "client_email": "adam.clay@compass.com",
    "meeting_timestamp": "2024-03-26 11:00:00",
    "meeting_description": "Compass - Annual Credit Facility Review Meeting",
}

In [ ]:
# Email prompt summary should be updated in the file
# currensee/agents/tools/outlook_tools.py in the function produce_client_email_summary

In [ ]:
# Finance news prompt summary should be updated in the file
# currensee/agents/tools/finance_tools.py in the summarize_finance_outputs function

In [ ]:
result = compiled_graph.invoke(init_state)

In [ ]:
summary = result["final_summary"]

# === Add the macro snapshot
macro_table = generate_macro_table()

In [ ]:
full_report = summary + "\n\n### Macro Financial Snapshot\n\n" + macro_table
print(full_report)

In [ ]:
# Traceback

client_industry_summary = result.get("client_industry_summary", [])
macro_news_summary = result.get("macro_news_summary", [])


# Function to format and print client_industry and macro_news summaries
def print_summary(summary, summary_name):
    print(f"\n{summary_name}:\n{'='*len(summary_name)}")
    for article in summary:
        title = article.get("title", "No Title")
        snippet = article.get("snippet", "No Snippet")
        date = article.get("date", "No Date")
        link = article.get("link", "")

        # Extract the source from the domain of the link
        source = link.split("/")[2] if link else "No Source"

        # Format and print the information
        print(f"Title: {title}")
        print(f"Snippet: {snippet}")
        print(f"Date: {date}")
        print(f"Source: {source}")
        print(f"Link: {link}")  # Displaying the full link
        print("-" * 40)  # Just a separator for readability


# Function to format and print the holdings summary
def print_holdings_summary(holdings_summary):
    print("\nClient Holdings Summary:")
    print("========================")

    for holding, articles in holdings_summary.items():
        print(f"\n--- {holding} ---")
        if not articles:
            print("No news found.")
            continue

        for article in articles:
            title = article.get("title", "No Title")
            snippet = article.get("snippet", "No Snippet")
            date = article.get("date", "No Date")
            link = article.get("link", "")
            source = link.split("/")[2] if link else "No Source"

            print(f"Title: {title}")
            print(f"Snippet: {snippet}")
            print(f"Date: {date}")
            print(f"Source: {source}")
            print(f"Link: {link}")
            print("-" * 40)


# Print the formatted summaries
print_summary(client_industry_summary, "Client Industry Summary")
print_summary(macro_news_summary, "Macro News Summary")
print_holdings_summary(result.get("client_holdings_summary", {}))

In [ ]:
# Retrieving specific figures within the summary

import re
from difflib import SequenceMatcher


# Extract facts from full_report — number + context window
def extract_facts_with_context(text, window=8):
    tokens = text.split()
    facts = []

    for i, word in enumerate(tokens):
        if re.match(r"\$?\d[\d,\.]*(%| billion| million|B|M|€)?\+?", word):
            start = max(i - window, 0)
            end = min(i + window + 1, len(tokens))
            context_phrase = " ".join(tokens[start:end])
            facts.append(
                {"value": word.replace(",", "").strip(), "phrase": context_phrase}
            )

    return facts


# Extract facts from article snippets with the same structure
def extract_snippet_facts(summary):
    snippet_facts = []

    for article in summary:
        snippet = article.get("snippet", "")
        title = article.get("title", "")
        link = article.get("link", "")
        source = article.get("source", "")

        tokens = snippet.split()
        for i, word in enumerate(tokens):
            if re.match(r"\$?\d[\d,\.]*(%| billion| million|B|M|€)?\+?", word):
                start = max(i - 8, 0)
                end = min(i + 8 + 1, len(tokens))
                context = " ".join(tokens[start:end])
                snippet_facts.append(
                    {
                        "value": word.replace(",", "").strip(),
                        "phrase": context,
                        "snippet": snippet,
                        "title": title,
                        "link": link,
                        "source": source,
                    }
                )

    return snippet_facts


# Find best match using similarity on phrases
def find_best_matches(report_facts, snippet_facts, threshold=0.4):
    matches = []

    for rf in report_facts:
        best_match = None
        best_score = 0

        for sf in snippet_facts:
            # Match by numeric value
            if rf["value"] in sf["value"] or sf["value"] in rf["value"]:
                score = SequenceMatcher(None, rf["phrase"], sf["phrase"]).ratio()
                if score > best_score:
                    best_score = score
                    best_match = sf

        if best_match and best_score > threshold:
            matches.append(
                {
                    "fact": rf["phrase"],
                    "match_score": round(best_score, 3),
                    "source": best_match["source"],
                    "title": best_match["title"],
                    "snippet": best_match["snippet"],
                    "link": best_match["link"],
                }
            )

    return matches


# Run matching
report_facts = extract_facts_with_context(full_report)
snippet_facts = extract_snippet_facts(
    client_industry_summary
    + macro_news_summary
    + [
        a
        for articles in result.get("client_holdings_summary", {}).values()
        for a in articles
    ]
)

matches = find_best_matches(report_facts, snippet_facts)

# Display results
for m in matches:
    print(f"Fact: {m['fact']}")
    print(f"Matched Snippet: {m['snippet']}")
    print(f"Match Score: {m['match_score']}")
    print(f"Source: {m['source']}")
    print(f"Title: {m['title']}")
    print(f"Link: {m['link']}")
    print("-" * 100)